## Importing Necessary Libraries

In [27]:
import tensorflow as tf

2024-06-05 10:49:02.691390: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 10:49:02.743167: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 10:49:03.458233: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [28]:
import warnings
warnings.filterwarnings('ignore')
import os, evaluate, torch, accelerate, nltk, re, datetime 
import pandas as pd
from collections import Counter
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForTokenClassification, AutoModelForCausalLM
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
import seaborn as sns
from pynvml import *
from numba import cuda
import matplotlib.pyplot as plt

## Get Current Working Directory

In [29]:
my_dir = os.getcwd() + '/' 
print(f"CWD: {my_dir}")

CWD: /home/sharjeelahmed/datascience/


## GPU Configuration

<p> Reset Cuda Memory</p>

In [30]:
device = cuda.get_current_device()
device.reset()
with torch.no_grad():
    torch.cuda.empty_cache()

RuntimeError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


<p>Check GPU Memory</p>

In [31]:
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'Total    : {info.total//1024**2} MB')
print(f'Free     : {info.free//1024**2} MB')
print(f'Used     : {info.used//1024**2} MB')

Total    : 15360 MB
Free     : 14490 MB
Used     : 869 MB


### Get Cuda Device

In [35]:
devices = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
devices = "cpu"

## Load Model and Tokenizer

In [36]:
tokenizer = AutoTokenizer.from_pretrained("ruslanmv/Medical-Llama3-8B")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
pre_trained_model = AutoModelForCausalLM.from_pretrained("ruslanmv/Medical-Llama3-8B").to(devices)  # If using GPU

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Model Inference

In [42]:
def askme(question):
    sys_message = ''' 
    You are an AI Medical Assistant trained on a vast dataset of health information. Please be thorough and
    provide an informative answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''
    
    # Create messages structured for the chat template
    messages = [{"role": "system", "content": sys_message}, {"role": "user", "content": question}]

    # Applying chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(devices)
    outputs = pre_trained_model.generate(**inputs, max_new_tokens=300, use_cache=True)  # Adjust max_new_tokens for longer responses

    # Extract and return the generated text
    answer = tokenizer.batch_decode(outputs)[0].strip()
    answer = answer.replace(sys_message, "").replace(question, "")
    answer = answer.split("assistant")[-1].strip()
    return answer

# Example usage
# - Context: First describe your problem.
# - Question: Then make the question.
question = '''
I'm a 35-year-old male and for the past few months, I've been experiencing fatigue, increased sensitivity to cold, and dry, itchy skin.  

Could these symptoms be related to dengue? 
'''
result = askme(question)
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Hi there! I'm here to help. Let's start by discussing your symptoms. 

    Fatigue: This could be a sign of several conditions, including dengue. However, it's important to note that fatigue can also be caused by other factors, such as stress, lack of sleep, or an underlying medical condition. 

    Increased sensitivity to cold: This could be a sign of dengue, as it's a common symptom of the disease. However, it's important to note that this symptom can also be caused by other factors, such as anemia or hypothyroidism. 

    Dry, itchy skin: This could be a sign of dengue, as it's a common symptom of the disease. However, it's important to note that this symptom can also be caused by other factors, such as eczema or psoriasis. 

    Based on your symptoms, it's possible that you may have dengue. However, it's important to note that these symptoms can also be caused by other factors. It's always best to consult with a healthcare professional to determine the underlying cause of your sy